In [1]:
import numpy as np
import pandas as pd

In [2]:
from lightautoml.reader.gpu.cudf_reader import CudfReader
from lightautoml.reader.base import PandasToPandasReader

from lightautoml.transformers.base import SequentialTransformer

from lightautoml.pipelines.utils import get_columns_by_role

from lightautoml.transformers.gpu import numeric_gpu, categorical_gpu, datetime_gpu
from lightautoml.transformers import numeric, categorical, datetime

from lightautoml.tasks import Task
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML

import pandas as pd
import time
from sklearn.model_selection import train_test_split

from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import cudf

In [3]:
features = pd.read_csv('../../data/small_new/multilabel/train Data.csv')
labels = pd.read_csv('../../data/small_new/multilabel/train labels.csv')
labels.drop(columns='id', inplace=True)
data = pd.concat([features, labels], axis=1)
data = data[['n_0000','n_0001','n_0002','n_0003',
             'n_0004','n_0005','n_0006','n_0007',
             'service_a', 'service_b', 'service_c', 
             'service_d', 'service_e', 'service_f',
             'service_g', 'service_h', 'service_i',
             'service_j', 'service_k', 'service_l',
             'service_m', 'service_n']]

tr_data, te_data = train_test_split(
    data, 
    test_size=0.2,  
    random_state=42
)

roles = {
    "target": {'service_a', 'service_b', 'service_c',
               'service_d', 'service_e', 'service_f',
               'service_g', 'service_h', 'service_i',
               'service_j', 'service_k', 'service_l',
               'service_m', 'service_n'},
    "drop" : ['id']
}

/tmp/ipykernel_18644/705777730.py:1: DtypeWarning: Columns (329,331,333,336,338,344,345,346,348,354,355,356,357,358,361,362,364,367,372,377,380,383,385,387,390,392,399,400,406,408,409,413,416,418,419,431,433,437,438,442,448,449,450,453,457,464,473,478,479,481,483,485,486,489,492,495,496,497,498,499,500,503,507,508,510,511,514,515,517,519,520,521,522,523,524,526,527,530,534,537,538,539,541,544,547,548,549,550,551,557,558,560,564,569,572,573,576,577,579,583,596,597,598,600,601,602,604,605,606,608,609,613,619,620,625,627,628,629,631,632,633,636,641,642,643,645,646,647,648,651,655,661,662,665,668,675,676,679,682,685,687,689,690,691,694,698,701,702,703,706,711,712,713,719,720,721,733,735,737,738,742,746,747,748,749,752,754,755,760,764,768,770,781,782,789,797,807,812,814,817,818,822,823,824,825,832,840,843,844,845,850,853,857,858,861,867,868,873,874,876,877,879,880,881,883,886,890,893,897,899,900,901,902,904,905,908,909,910,912,913,914,915,916,922,923,931,933,935,937,939,942,943,946,951,955,

In [4]:
adv_roles = True

## Imports (for potential use)

In [5]:
# Imports from our package
from lightautoml.automl.base import AutoML

from lightautoml.automl.presets.gpu.tabular_gpu_presets import TabularAutoMLGPU, TabularUtilizedAutoMLGPU
from lightautoml.tasks import Task

from lightautoml.pipelines.features.gpu.lgb_pipeline_gpu import LGBSimpleFeaturesGPU, LGBAdvancedPipelineGPU
from lightautoml.pipelines.features.gpu.linear_pipeline_gpu import LinearFeaturesGPU

from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml.pipelines.features.linear_pipeline import LinearFeatures


from lightautoml.ml_algo.gpu.boost_cb_gpu import BoostCBGPU
from lightautoml.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml.ml_algo.gpu.linear_gpu import LinearLBFGSGPU

from lightautoml.ml_algo.boost_cb import BoostCB
from lightautoml.ml_algo.linear_sklearn import LinearLBFGS


from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector

## TabularAutoML

In [6]:
task = Task('multilabel')

multilabel isn`t supported in lgb


In [7]:
automl = TabularAutoML(
    task = task, 
    timeout = 3600,
    cpu_limit = 4,
    reader_params = {'n_jobs': 4, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'cb']]}
)

In [8]:
oof_pred = automl.fit_predict(data, roles = roles, verbose = 2)

[15:33:43] Stdout logging level is INFO2.
[15:33:43] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[15:33:43] Task: multilabel

[15:33:43] Start automl preset with listed constraints:
[15:33:43] - time: 3600.00 seconds
[15:33:43] - CPU: 4 cores
[15:33:43] - memory: 16 GB

[15:33:43] Train data shape: (14644, 22)



/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[15:33:46] Feats was rejected during automatic roles guess: []
[15:33:46] Layer 1 train process start. Time left 3596.44 secs


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[15:33:46] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[15:33:46] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[15:33:49] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[15:33:51] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[15:33:53] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.350364600194018
[15:33:53] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[15:33:53] Time left 3589.44 secs



...

[15:34:10] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[15:34:20] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[15:34:30] Fitting Lvl_0_Pipe_1_Mod_0_CatBoost finished. score = -7.322678702153068
[15:34:30] Lvl_0_Pipe_1_Mod_0_CatBoost fitting and predicting completed
[15:34:30] Time left 3552.29 secs

[15:34:30] Layer 1 training completed.

[15:34:30] Blending: optimization starts with equal weights and score -7.316146249899412
[15:34:31] Blending: iteration 0: score = -7.315961742736511, weights = [0.5746675 0.4253325]
[15:34:31] Blending: iteration 1: score = -7.315961742736511, weights = [0.5746675 0.4253325]
[15:34:31] Blending: no score update. Terminated

[15:34:31] Automl preset training completed in 48.12 seconds

[15:34:31] Model description:
Final prediction for new objects (level 0) = 
	 0.57467 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.42533 * (3 averaged models Lvl_0_Pipe_1_Mod_0_CatBoost) 



In [7]:
task = Task('multilabel', device='gpu')

multilabel isn`t supported in lgb


In [8]:
automl_gpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['pb', 'pb_tuned']]}
)

In [9]:
oof_pred_gpu = automl_gpu.fit_predict(data, roles = roles, verbose = 2)

[13:40:32] Stdout logging level is INFO2.
[13:40:32] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[13:40:32] Task: multilabel

[13:40:32] Start automl preset with listed constraints:
[13:40:32] - time: 3600.00 seconds
[13:40:32] - CPU: 1 cores
[13:40:32] - memory: 16 GB

[13:40:32] Train data shape: (14644, 22)
Feats was rejected during automatic roles guess: []
[13:40:34] Layer 1 train process start. Time left 3598.38 secs
[13:40:55] Selector_XGB fitting and predicting completed
[13:40:55] Start fitting Lvl_0_Pipe_0_Mod_0_PB ...
[13:40:55] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_PB (orig) =====
[13:40:57] Stdout logging level is INFO.
[13:40:57] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:40:58] Iter 0; Sample 0, BCE = 0.3789424276385505; 
[13:40:58] Iter 100; Sample 0, BCE = 0.35316098700961457; 
[13:40:59] Iter 200; Sample 0, BCE = 0.35308152496791867; 
[13:40:59] Iter 300; Sample 0, BCE = 0.353

[13:41:31] Iter 100; Sample 0, BCE = 0.3547875670015221; 
[13:41:32] Iter 200; Sample 0, BCE = 0.35279406745567377; 
[13:41:32] Iter 300; Sample 0, BCE = 0.3524260047308684; 
[13:41:32] Iter 400; Sample 0, BCE = 0.3523019752745216; 
[13:41:33] Iter 500; Sample 0, BCE = 0.3523465918328254; 
[13:41:33] Iter 600; Sample 0, BCE = 0.35243949489674004; 
[13:41:33] Early stopping at iter 627, best iter 427, best_score 0.3522842078653014
[13:41:33] Stdout logging level is INFO.
[13:41:33] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:41:33] Iter 0; Sample 0, BCE = 0.37929389701453603; 
[13:41:34] Iter 100; Sample 0, BCE = 0.3537212303618661; 
[13:41:34] Iter 200; Sample 0, BCE = 0.3523936308820458; 
[13:41:35] Iter 300; Sample 0, BCE = 0.3523387433086477; 
[13:41:35] Iter 400; Sample 0, BCE = 0.35249046995321504; 
[13:41:35] Early stopping at iter 455, best iter 255, best_score 0.35228731166978755
[13:41:35] Stdout logging level is INFO.
[13:41:35] GDBT train starts. Max iter

[13:42:08] Iter 0; Sample 0, BCE = 0.3794120082671062; 
[13:42:09] Iter 100; Sample 0, BCE = 0.3549137037341633; 
[13:42:09] Iter 200; Sample 0, BCE = 0.35274286342711714; 
[13:42:09] Iter 300; Sample 0, BCE = 0.3522570424169081; 
[13:42:10] Iter 400; Sample 0, BCE = 0.35212915780069526; 
[13:42:10] Iter 500; Sample 0, BCE = 0.35212918003596694; 
[13:42:10] Iter 600; Sample 0, BCE = 0.3522037441256741; 
[13:42:11] Early stopping at iter 675, best iter 475, best_score 0.35210327812358483
[13:42:11] Stdout logging level is INFO.
[13:42:11] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:42:11] Iter 0; Sample 0, BCE = 0.37906035280929584; 
[13:42:12] Iter 100; Sample 0, BCE = 0.3535995602508987; 
[13:42:13] Iter 200; Sample 0, BCE = 0.3539212128991216; 
[13:42:13] Iter 300; Sample 0, BCE = 0.3549163979089165; 
[13:42:13] Early stopping at iter 318, best iter 118, best_score 0.35349410655816055
[13:42:14] Stdout logging level is INFO.
[13:42:14] GDBT train starts. Max iter 

[13:42:44] Early stopping at iter 680, best iter 480, best_score 0.35211480727351385
[13:42:45] Stdout logging level is INFO.
[13:42:45] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:42:45] Iter 0; Sample 0, BCE = 0.37941098618416447; 
[13:42:45] Iter 100; Sample 0, BCE = 0.35490669347519876; 
[13:42:45] Iter 200; Sample 0, BCE = 0.3527619651981761; 
[13:42:46] Iter 300; Sample 0, BCE = 0.35229019910355813; 
[13:42:46] Iter 400; Sample 0, BCE = 0.35217539904144474; 
[13:42:46] Iter 500; Sample 0, BCE = 0.35215955434096313; 
[13:42:47] Iter 600; Sample 0, BCE = 0.3522079793505275; 
[13:42:47] Early stopping at iter 675, best iter 475, best_score 0.35213785598743047
[13:42:47] Stdout logging level is INFO.
[13:42:47] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:42:47] Iter 0; Sample 0, BCE = 0.37939156554883746; 
[13:42:48] Iter 100; Sample 0, BCE = 0.3547736968873892; 
[13:42:48] Iter 200; Sample 0, BCE = 0.35271303090202627; 
[13:42:48] Iter 300; Sa

[13:43:18] Iter 500; Sample 0, BCE = 0.3521802067582093; 
[13:43:19] Iter 600; Sample 0, BCE = 0.3521466588779485; 
[13:43:19] Iter 700; Sample 0, BCE = 0.35216636467354573; 
[13:43:19] Iter 800; Sample 0, BCE = 0.35223121551637676; 
[13:43:19] Early stopping at iter 836, best iter 636, best_score 0.3521335402000663
[13:43:20] Stdout logging level is INFO.
[13:43:20] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:43:20] Iter 0; Sample 0, BCE = 0.3792923634282585; 
[13:43:20] Iter 100; Sample 0, BCE = 0.3537795205045072; 
[13:43:21] Iter 200; Sample 0, BCE = 0.35243684415836535; 
[13:43:21] Iter 300; Sample 0, BCE = 0.35243983682645375; 
[13:43:22] Iter 400; Sample 0, BCE = 0.3525660507905294; 
[13:43:22] Early stopping at iter 477, best iter 277, best_score 0.3523858249066567
[13:43:22] Stdout logging level is INFO.
[13:43:22] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:43:22] Iter 0; Sample 0, BCE = 0.3792093747447789; 
[13:43:23] Iter 100; Sample 

[13:43:55] Iter 0; Sample 0, BCE = 0.3794151938785409; 
[13:43:55] Iter 100; Sample 0, BCE = 0.35497275069697376; 
[13:43:56] Iter 200; Sample 0, BCE = 0.35277768998699377; 
[13:43:56] Iter 300; Sample 0, BCE = 0.3523201744059999; 
[13:43:56] Iter 400; Sample 0, BCE = 0.3521741530390748; 
[13:43:57] Iter 500; Sample 0, BCE = 0.3521575966407192; 
[13:43:57] Iter 600; Sample 0, BCE = 0.3522006682688433; 
[13:43:57] Early stopping at iter 675, best iter 475, best_score 0.35212803453728697
[13:43:58] Stdout logging level is INFO.
[13:43:58] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:43:58] Iter 0; Sample 0, BCE = 0.3794078709504119; 
[13:43:58] Iter 100; Sample 0, BCE = 0.3548908296497685; 
[13:43:58] Iter 200; Sample 0, BCE = 0.3527822072659085; 
[13:43:59] Iter 300; Sample 0, BCE = 0.3523227610222962; 
[13:43:59] Iter 400; Sample 0, BCE = 0.3522248593968491; 
[13:43:59] Iter 500; Sample 0, BCE = 0.35223345666448314; 
[13:44:00] Iter 600; Sample 0, BCE = 0.35226363757

[13:44:30] Iter 0; Sample 0, BCE = 0.3794238176556505; 
[13:44:30] Iter 100; Sample 0, BCE = 0.3550975330516116; 
[13:44:30] Iter 200; Sample 0, BCE = 0.3528039307834386; 
[13:44:30] Iter 300; Sample 0, BCE = 0.35230134154291065; 
[13:44:31] Iter 400; Sample 0, BCE = 0.35216306705533745; 
[13:44:31] Iter 500; Sample 0, BCE = 0.35215303947576815; 
[13:44:31] Iter 600; Sample 0, BCE = 0.3521586973684932; 
[13:44:32] Iter 700; Sample 0, BCE = 0.3522116982135713; 
[13:44:32] Early stopping at iter 737, best iter 537, best_score 0.3521303048187547
[13:44:32] Stdout logging level is INFO.
[13:44:32] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:44:32] Iter 0; Sample 0, BCE = 0.37941491379394154; 
[13:44:33] Iter 100; Sample 0, BCE = 0.3549531110061637; 
[13:44:33] Iter 200; Sample 0, BCE = 0.35275457900511287; 
[13:44:33] Iter 300; Sample 0, BCE = 0.3523125425546641; 
[13:44:34] Iter 400; Sample 0, BCE = 0.3521672000912071; 
[13:44:34] Iter 500; Sample 0, BCE = 0.3521405751

[13:45:05] Iter 200; Sample 0, BCE = 0.3527147865356635; 
[13:45:05] Iter 300; Sample 0, BCE = 0.3523112715593415; 
[13:45:05] Iter 400; Sample 0, BCE = 0.35218546423344427; 
[13:45:06] Iter 500; Sample 0, BCE = 0.35223649141389324; 
[13:45:06] Iter 600; Sample 0, BCE = 0.3522890981652396; 
[13:45:06] Early stopping at iter 622, best iter 422, best_score 0.35217644301577905
[13:45:06] Stdout logging level is INFO.
[13:45:06] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:45:06] Iter 0; Sample 0, BCE = 0.3794185703964684; 
[13:45:07] Iter 100; Sample 0, BCE = 0.3550140332560637; 
[13:45:07] Iter 200; Sample 0, BCE = 0.35277195705400044; 
[13:45:08] Iter 300; Sample 0, BCE = 0.35229863211829954; 
[13:45:08] Iter 400; Sample 0, BCE = 0.35213845971884994; 
[13:45:08] Iter 500; Sample 0, BCE = 0.35210977373578245; 
[13:45:09] Iter 600; Sample 0, BCE = 0.3521246705515924; 
[13:45:09] Iter 700; Sample 0, BCE = 0.3521950840180166; 
[13:45:09] Early stopping at iter 765, best i

In [12]:
gpu_inf = automl_gpu.predict(data)

In [13]:
automl_gpu.to_cpu()

TabularAutoMLGPU
{'_binary': False, 'data_size': 15, 'categorical_idx': {'int': [12, 13, 14], 'str': ['le__n_0000', 'le__n_0003', 'inter__(n_0000__n_0003)']}, 'embed_sizes': array([ 2, 12, 12], dtype=int32), 'output_size': 14, 'cs': [1e-05, 5e-05, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000], 'max_iter': 100, 'tol': 1e-06, 'early_stopping': 2, 'loss': TorchLossWrapper(
  (base_loss): BCELoss()
), 'metric': <lightautoml.tasks.losses.base.MetricFunc object at 0x7f62a113f670>, 'model': CatMulticlass(
  (linear): Linear(in_features=12, out_features=14, bias=False)
  (softmax): Softmax(dim=1)
)}
TorchBasedLogisticRegression
{'_name': 'multilabel', 'device': 'gpu', 'losses': {'lgb': <lightautoml.tasks.losses.lgb.LGBLoss object at 0x7f62a0e673a0>, 'sklearn': <lightautoml.tasks.losses.sklearn.SKLoss object at 0x7f62a0e67190>, 'torch': <lightautoml.tasks.losses.torch.TORCHLoss object at 0x7f62a113f040>, 'cb': <lightautoml.tasks.lo

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

multilabel isn`t supported in lgb
[15:35:24] CatBoost uses as obj. MultiCrossEntropy.


In [14]:
cpu_inf = automl_gpu.predict(data)

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [15]:
cluster = LocalCUDACluster(rmm_managed_memory=True, CUDA_VISIBLE_DEVICES="0",
                               protocol="ucx", enable_nvlink=True,
                               memory_limit="8GB")
print("dashboard:", cluster.dashboard_link)
client = Client(cluster)
client.run(cudf.set_allocator, "managed")

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/comm/ucx.py:61: UserWarning: A CUDA context for device 0 already exists on process ID 21513. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
2022-11-30 15:35:26,754 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-wf7z6oqf', purging
2022-11-30 15:35:26,754 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-11-30 15:35:26,754 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


dashboard: http://127.0.0.1:8787/status


{'ucx://127.0.0.1:60617': None}

In [16]:
task = Task('multilabel', device='mgpu')

automl_mgpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42, 'npartitions': 2},
    general_params = {'use_algos': [['xgb', 'linear_l2']]},
    client = client
)

oof_pred_mgpu = automl_mgpu.fit_predict(data, roles = roles, verbose = 2)

multilabel isn`t supported in lgb
[15:35:27] CatBoost uses as obj. MultiCrossEntropy.
[15:35:27] Stdout logging level is INFO2.
[15:35:27] Task: multilabel

[15:35:27] Start automl preset with listed constraints:
[15:35:27] - time: 3600.00 seconds
[15:35:27] - CPU: 1 cores
[15:35:27] - memory: 16 GB

[15:35:27] Train data shape: (14644, 22)
Feats was rejected during automatic roles guess: []
[15:35:29] Layer 1 train process start. Time left 3598.23 secs
[15:35:29] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[15:35:29] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
Score: -7.597461223602295
Score: -7.577960014343262
Score: -7.561263084411621
Score: -7.507802963256836
Score: -7.484713554382324
Score: -7.434474468231201
Score: -7.434474468231201
Score: -7.409946441650391
Score: -7.409946441650391
Score: -7.409946441650391
[15:35:33] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
Score: -7.4780988693237305
Score: -7.458440780639

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[15:35:50] task [xgboost.dask-0]:ucx://127.0.0.1:60617 got new rank 0


[15:36:15] Selector_XGB fitting and predicting completed
[15:36:15] Start fitting Lvl_0_Pipe_1_Mod_0_XGB ...
[15:36:15] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_0_XGB (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[15:36:16] task [xgboost.dask-0]:ucx://127.0.0.1:60617 got new rank 0


[15:36:43] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_XGB (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[15:36:43] task [xgboost.dask-0]:ucx://127.0.0.1:60617 got new rank 0


[15:37:11] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_XGB (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[15:37:11] task [xgboost.dask-0]:ucx://127.0.0.1:60617 got new rank 0


[15:37:39] Lvl_0_Pipe_1_Mod_0_XGB fitting and predicting completed
[15:37:39] Time left 3467.91 secs

[15:37:39] Layer 1 training completed.

[15:37:39] Blending: Optimization starts with equal weights and score -7.369040055931575
[15:37:39] Blending, iter 0: score = -7.332288272275898, weights = [0.90295106 0.09704894]
[15:37:39] Blending, iter 1: score = -7.332288272275898, weights = [0.90295106 0.09704894]
[15:37:39] No score update. Terminated
[15:37:39] Automl preset training completed in 132.30 seconds

[15:37:39] Model description:
Final prediction for new objects (level 0) = 
	 0.90295 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.09705 * (3 averaged models Lvl_0_Pipe_1_Mod_0_XGB) 



In [17]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
print(oof_pred.data.T)
print()
print(oof_pred_mgpu.data.T)

[[0.5572413  0.5140602  0.554345   ... 0.32615978 0.55932724 0.5283271 ]
 [0.10613684 0.15360174 0.10628839 ... 0.24034458 0.10660344 0.13533857]
 [0.01066768 0.01306654 0.01075604 ... 0.00172346 0.01067023 0.00936168]
 ...
 [0.0912509  0.11758646 0.09017038 ... 0.2209489  0.08924023 0.12009337]
 [0.18382518 0.22122423 0.17709294 ... 0.3786014  0.16766214 0.22966513]
 [0.02809653 0.04665985 0.02616439 ... 0.04066981 0.02415165 0.05030812]]

[[0.56842893 0.5196632  0.5663695  ... 0.32545125 0.5775144  0.5322845 ]
 [0.10147692 0.16129875 0.10777053 ... 0.24136236 0.10288282 0.12789753]
 [0.01173552 0.01505887 0.01187614 ... 0.00313549 0.00997119 0.00903109]
 ...
 [0.09023867 0.1269161  0.08745717 ... 0.24356261 0.07901268 0.11170406]
 [0.18878452 0.23331442 0.17551136 ... 0.39841276 0.15086576 0.21826142]
 [0.0252515  0.05333086 0.02523747 ... 0.04038102 0.01980539 0.04349071]]

[[0.5664643  0.5166605  0.5704907  ... 0.33915734 0.57409036 0.5306796 ]
 [0.09986173 0.16810127 0.10628133 ..